## GPU vs CPU performance (pairwise acceleration)

In [ ]:
import time
import numpy as _np
try:
    import cupy as _cp
    HAVE_CUPY = True
except Exception:
    HAVE_CUPY = False

def bench_pairwise(N=800):
    from importlib import reload
    import nbody_adv as nb
    # CPU (NumPy)
    x_cpu = _np.random.RandomState(0).normal(0, 1, size=(N,3)).astype(float)
    m_cpu = _np.ones(N, dtype=float)
    t0 = time.perf_counter()
    _ = nb.pairwise_accel(x_cpu, m_cpu, eps=1e-2)
    t1 = time.perf_counter()
    cpu_ms = (t1 - t0)*1e3
    gpu_ms = None
    if HAVE_CUPY:
        import cupy as cp
        x_gpu = cp.asarray(x_cpu); m_gpu = cp.asarray(m_cpu)
        _ = nb.pairwise_accel(x_gpu, m_gpu, eps=1e-2)  # warmup
        cp.cuda.Stream.null.synchronize()
        t2 = time.perf_counter()
        _ = nb.pairwise_accel(x_gpu, m_gpu, eps=1e-2)
        cp.cuda.Stream.null.synchronize()
        t3 = time.perf_counter()
        gpu_ms = (t3 - t2)*1e3
    return cpu_ms, gpu_ms

cpu_ms, gpu_ms = bench_pairwise(N=800)
cpu_ms, gpu_ms

### Barycenter drift (Scenario A)

In [ ]:
import numpy as np
from importlib import reload; import nbody_adv as nb; reload(nb)
# Re-run short binary
x, v, m = nb.circular_binary_ic(1.0e22, 1.0e22, 1.0e7)
snapB = nb.leapfrog(x, v, m, dt=0.25, steps=2000, eps=1e3, record_every=20)
# Track barycenter
R = []; 
for s in snapB:
    Rc, Vc = nb.barycenter(s['x'], s['v'], m)
    R.append(np.linalg.norm(Rc))
float(max(R))

## Scenario C — Plummer cluster (toy initializer)

In [ ]:
from importlib import reload; import nbody_adv as nb; reload(nb)
N=200; Mtot=2e25; a=5e8
x, v, m = nb.plummer_sphere(N, Mtot, a, seed=7)
out = nb.leapfrog(x, v, m, dt=5.0, steps=2000, eps=1e6, record_every=20)
len(out)

In [ ]:
import numpy as np, matplotlib.pyplot as plt
X = np.array([s['x'] for s in out])
plt.figure(figsize=(6,6))
plt.plot(X[:, :, 0], X[:, :, 1], '.', markersize=1)
plt.gca().set_aspect('equal','box'); plt.title('Plummer cluster (projection)')
plt.show()